Here we can supply the Apple credentials if I haven't stored them in my keyring yet:

In [ ]:
import os
import sys

# picked up this trick from alexleake https://github.com/alexleake/pyicloud_example/blob/master/gps_location.py
if os.path.isfile('./credentials.py'):
    from credentials import APPLE_ID, APPLE_PASSWORD
else:
    APPLE_ID = 'REPLACE@ME.COM'
    APPLE_PASSWORD = 'REPLACE_ME'

This method of using PyiCloud authentication doesn't quite work:

In [ ]:
# Py3.5 pattern from http://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path
import importlib.util
spec = importlib.util.spec_from_file_location("pyicloud", "../pyicloud/pyicloud/__init__.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
api = foo.PyiCloudService(APPLE_ID, APPLE_PASSWORD)

This method of using PyiCloud authentication appears to work consistently:

In [ ]:
# Py3.3/3.4 pattern from pattern from http://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path
from importlib.machinery import SourceFileLoader

foo = SourceFileLoader("pyicloud", "/Users/mike/code/pyicloud/pyicloud/__init__.py").load_module()
api = foo.PyiCloudService(APPLE_ID, APPLE_PASSWORD)

And here's the add-on code (supplied by picklepete) that I need to successfully get through 2-factor authentication:

In [ ]:
#Two-factor authentication code appropriated from https://github.com/picklepete/pyicloud/blob/master/README.rst
if api.requires_2fa:
    import click
    print("Two-factor authentication required. Your trusted devices are:")

    devices = api.trusted_devices
    for i, device in enumerate(devices):
        print("  %s: %s" % (i, device.get('deviceName',
            "SMS to %s" % device.get('phoneNumber'))))

    device = click.prompt('Which device would you like to use?', default=0)
    device = devices[device]
    if not api.send_verification_code(device):
        print("Failed to send verification code")
        sys.exit(1)

    code = click.prompt('Please enter validation code (probably the one sent to your phone, not the one displayed on your Mac, if you\'re seeing both):')
    if not api.validate_verification_code(device, code):
        print("Failed to verify verification code")
        sys.exit(1)

Now to demonstrate that I can access the Contacts data:

In [ ]:
for c in api.contacts.all():
    print(c.get('firstName'), c.get('lastName'), c.get('companyName'), c.get('phones'))